In [21]:
%cd drive/MyDrive/Complexity-Measure/

/content/drive/MyDrive/Complexity-Measure


In [6]:
from tensorflow.keras.applications import VGG16, Xception, ResNet50V2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
import numpy as np
import cv2
from tensorflow.keras.datasets import mnist

In [7]:
from keras import backend as K
K.clear_session()

In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape, y_train.shape

11493376/11490434 [==============================] - 0s 0us/step


((60000, 28, 28), (60000,))

In [9]:
def one_to_three_dim(x_train, dim):
  img_lst = []
  for i in range(len(x_train)):
    img = cv2.resize(x_train[i], dim, interpolation = cv2.INTER_AREA) 
    img = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    img_lst.append(img)
  rgb_arr = np.stack([img_lst],axis=4)
  rgb_arr_to_3d = np.squeeze(rgb_arr, axis=4)
  return rgb_arr_to_3d

In [19]:
def feature_extractor(images, model, x, y, z):
  feature_list = []
  for i in range(len(images)):
    img_array = np.expand_dims(images[i], axis=0)
    features = model.predict(img_array)
    feature_list.append(features)
  feat_lt = np.reshape(feature_list, (-1, x*y*z))
  return feat_lt

In [11]:
dim = (72, 72)
rgb_list = one_to_three_dim(x_train, dim)
print(rgb_list.shape)

(60000, 72, 72, 3)


In [13]:
vgg = VGG16(weights='imagenet',
            include_top=False,
            input_shape=(72, 72, 3))

for layers in vgg.layers:
  layers.trainable=False

print(vgg.output)


58892288/58889256 [==============================] - 0s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, 2, 2, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")


In [20]:
vgg_features = feature_extractor(rgb_list[0:1000, :, :, :], vgg, 2, 2, 512)
np.save("vgg16_features.npy", vgg_features)

In [22]:
np.save("vgg16_features.npy", vgg_features)
# feat_lt = np.load("vgg16_features.npy")

In [14]:
xception = Xception(weights='imagenet',
            include_top=False,
            input_shape=(72, 72, 3))

for layers in xception.layers:
  layers.trainable=False

print(xception.output)

83689472/83683744 [==============================] - 1s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 3, 2048), dtype=tf.float32, name=None), name='block14_sepconv2_act/Relu:0', description="created by layer 'block14_sepconv2_act'")


In [23]:
xception_features = feature_extractor(rgb_list[0:1000, :, :, :], xception, 3, 3, 2048)
np.save("xception_features.npy", xception_features)

In [15]:
resnet = ResNet50V2(weights='imagenet',
            include_top=False,
            input_shape=(72, 72, 3))

for layers in resnet.layers:
  layers.trainable=False

print(resnet.output)

94674944/94668760 [==============================] - 1s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 3, 2048), dtype=tf.float32, name=None), name='post_relu/Relu:0', description="created by layer 'post_relu'")


In [24]:
resnet_features = feature_extractor(rgb_list[0:1000, :, :, :], resnet, 3, 3, 2048)
np.save("resnet_features.npy", resnet_features)

In [25]:
vgg_features.shape, xception_features.shape, resnet_features.shape

((1000, 2048), (1000, 18432), (1000, 18432))